# Nuisance Alarm Filter

## Documentation

Alarm events are found in AlarmList files and EventList consist of all events regardless of whether they are alarms or not. Hence, the alarm tagging must be done is post to tag which event in EventList is an alarm or not, in order to facillitate further analysis.

Steps Overview For Baseline Alarm Tagging:
1. Load all AlarmList files from the same batch into df_AL
2. Create a column called isAlarm in df_AL with the default value of True
3. Delete all columns in df_AL except for alarmID, SCS_time, environment and assetID
4. Load all EventList files from the same batch into df_AE
5. Perform a data join to merge in the column of isAlarm using alarmID, SCS_time, environment and EQUIPMENT_NAME, assetID as the reference keys
6. Delete df_AL
7. Replace all null values of isAlarm in df_AE with False

Steps Overview for Nuisance Alarm Tagging:
1. Index events by "SCS_TIME"
2. Consolidate "EVENT_DESCRIPTION" and "EVENT_STATUS" as a single field for value comparison as "eventValue_N"
3. Group by "eventValue_N" and "SCS_TIME" values using "ENVIRONMENT", "ASSET_ID_RAW", "EQUIPMENT_NAME" to extract the past 5 values as new columns
4. Check and tag cases with consecutive repeat values within a 1 min sliding window
5. Check and tag cases with a consecutive toggling pair within a 1 min sliding window (double and triple toggle)
6. Remove intermediate columns

Notes:
1. This script does not specifically covers of event toggling prior to an environment crash; additional finetuning would be required, though this serves as a starting base.
2. Event state changes to the user acknowledgement may also be treated as nuisance alarms if they occur closely together with an event with the same event description and category
3. A single asset can generate related events from one or more different AlarmIDs within a short period of time
4. Script has been updated to allow for either single file or multi file (subset) export or both

## Initialisation

### Load libraries

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import datetime as dt
import time

### Configure display

In [2]:
# Enable display of all columns for dataframes with many variables
pd.set_option('display.max_columns', None)

### Set Up Directory

In [3]:
# Check current directory location
import os
cwd = os.getcwd()
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\scripts'

In [4]:
# Define root file directory folder where the files are being stored
#os.chdir(cwd + alarmLoc)
os.chdir(os.path.dirname(os.getcwd()) + '\\alarm-event-logs')

# Check current directory location
cwd = os.getcwd()

# Check directory location
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [5]:
# Location of Alarm and Normal Event Files
#alarmLoc = '\\sample_data_ecs\\batch 001 - 20201230 to 20210201\\dataCleaned\\AlarmLog'
#eventLoc = '\\sample_data_ecs\\batch 001 - 20201230 to 20210201\\dataCleaned\\EventLog'
#alarmLoc = '\\sample_data_ats\\batch 001 - 20201230 to 20210201\\dataCleaned\\AlarmLog'
#eventLoc = '\\sample_data_ats\\batch 001 - 20201230 to 20210201\\dataCleaned\\EventLog'
alarmLoc = '\\sample_data_cms\\batch 001 - 20201230 to 20210201\\dataCleaned\\AlarmLog'
eventLoc = '\\sample_data_cms\\batch 001 - 20201230 to 20210201\\dataCleaned\\EventLog'

## Alarm Tagging

### Load AlarmList Files

In [6]:
# Define root file directory folder where the files are being stored
os.chdir(cwd + alarmLoc)
#os.chdir(cwd + eventLoc)
# Check directory location
os.getcwd()

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_cms\\batch 001 - 20201230 to 20210201\\dataCleaned\\AlarmLog'

In [7]:
# Gather list of cleaned alarmList files
dfs = [pd.read_csv(f)
        for f in os.listdir(os.getcwd()) if f.endswith('csv')]

# Compile list of cleaned alarmList files into a single dataframe
df_al = pd.concat(dfs, ignore_index=True).reset_index()

# Delete redundant index col
del df_al["index"]

# Print df summary stats
print(df_al.shape)
print(df_al.info())

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (5,22) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(836477, 23)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836477 entries, 0 to 836476
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ENTRY_CODE_SUFFIX         836477 non-null  object 
 1   ENTRY_CODE                836477 non-null  int64  
 2   ALARM_ID                  836477 non-null  int64  
 3   USER_ID                   836477 non-null  int64  
 4   EQUIPMENT_NAME            836473 non-null  object 
 5   VALUE                     836477 non-null  object 
 6   VALUE_STATE               836477 non-null  int64  
 7   ACKNOWLEDGEMENT_REQUIRED  836477 non-null  bool   
 8   SEVERITY                  836477 non-null  int64  
 9   HIDDEN                    836477 non-null  bool   
 10  THEME                     836477 non-null  int64  
 11  EQUIPMENT_DATE            836477 non-null  object 
 12  ACQUISITION_DATE          836477 non-null  object 
 13  SCS_TIME                  83647

In [8]:
# Inspect df
df_al.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS
0,=,4340,4340,0,:HGN:COMS:PISS_0001:PDC__0001:PIDS_0011,9,1,True,2,True,0,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,44,16,OCCCMS,1.0,COM/HGN/1/PID11,Ent1:LD33,PID Status,FAN ALARM,NaN
1,+,12365,12365,0,:PGL:BMF:Supervision:LENV_SMS_,1,1,True,4,True,0,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,60,19,OCCCMS,1.0,PGL_LENV_SMS_,STN SMS Server - Environment PGLSMS,Environment 1 Status,STOP,NaN
2,+,12366,12366,0,:PGL:BMF:Supervision:LENV_SMS_,1,1,True,4,True,0,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,60,19,OCCCMS,1.0,PGL_LENV_SMS_,STN SMS Server - Environment PGLSMS,Environment 2 Status,STOP,NaN
3,+,12367,12367,0,:SKG:BMF:Supervision:LENV_SMS_,1,1,True,4,True,0,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.230729216,60,18,OCCCMS,1.0,SKG_LENV_SMS_,STN SMS Server - Environment SKGSMS,Environment 1 Status,STOP,NaN
4,+,12368,12368,0,:SKG:BMF:Supervision:LENV_SMS_,1,1,True,4,True,0,2021-01-24 03:06:41.230729216,2021-01-24 03:06:41.230729216,2021-01-24 03:06:41.230729216,60,18,OCCCMS,1.0,SKG_LENV_SMS_,STN SMS Server - Environment SKGSMS,Environment 2 Status,STOP,NaN


In [9]:
# Junk Redundant Cols to save memory (update 1.4)
df_al = df_al[["ALARM_ID", "SCS_TIME", "ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW", "EVENT_DESCRIPTION", "EVENT_STATUS"]]

# Add in Alarm Tag Col
df_al["isAlarm"] = True

# Print df summary stats
print(df_al.shape)
print(df_al.info())

(836477, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836477 entries, 0 to 836476
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ALARM_ID           836477 non-null  int64 
 1   SCS_TIME           836477 non-null  object
 2   ENVIRONMENT        836477 non-null  object
 3   EQUIPMENT_NAME     836473 non-null  object
 4   ASSET_ID_RAW       836477 non-null  object
 5   EVENT_DESCRIPTION  836477 non-null  object
 6   EVENT_STATUS       836477 non-null  object
 7   isAlarm            836477 non-null  bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 45.5+ MB
None


In [10]:
# Inspect df
df_al.head()

,ALARM_ID,SCS_TIME,ENVIRONMENT,EQUIPMENT_NAME,ASSET_ID_RAW,EVENT_DESCRIPTION,EVENT_STATUS,isAlarm
0,4340,2021-01-24 03:06:40.528445952,OCCCMS,:HGN:COMS:PISS_0001:PDC__0001:PIDS_0011,COM/HGN/1/PID11,PID Status,FAN ALARM,True
1,12365,2021-01-24 03:06:41.324359936,OCCCMS,:PGL:BMF:Supervision:LENV_SMS_,PGL_LENV_SMS_,Environment 1 Status,STOP,True
2,12366,2021-01-24 03:06:41.324359936,OCCCMS,:PGL:BMF:Supervision:LENV_SMS_,PGL_LENV_SMS_,Environment 2 Status,STOP,True
3,12367,2021-01-24 03:06:41.230729216,OCCCMS,:SKG:BMF:Supervision:LENV_SMS_,SKG_LENV_SMS_,Environment 1 Status,STOP,True
4,12368,2021-01-24 03:06:41.230729216,OCCCMS,:SKG:BMF:Supervision:LENV_SMS_,SKG_LENV_SMS_,Environment 2 Status,STOP,True


### Load EventList Files

In [11]:
# Check directory location
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [12]:
# Define root file directory folder where the files are being stored
#os.chdir(cwd + alarmLoc)
os.chdir(cwd + eventLoc)
# Check directory location
os.getcwd()

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_cms\\batch 001 - 20201230 to 20210201\\dataCleaned\\EventLog'

In [13]:
# Gather list of cleaned alarmList files
dfs = [pd.read_csv(f)
        for f in os.listdir(os.getcwd()) if f.endswith('csv')]

# Compile list of cleaned alarmList files into a single dataframe
df_ae = pd.concat(dfs, ignore_index=True).reset_index()

# Delete redundant index col
del df_ae["index"]

# Print df summary stats
print(df_ae.shape)
print(df_ae.info())

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (5,22) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(19604630, 33)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19604630 entries, 0 to 19604629
Data columns (total 33 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ENTRY_CODE_SUFFIX         object 
 1   ENTRY_CODE                int64  
 2   ALARM_ID                  int64  
 3   USER_ID                   int64  
 4   EQUIPMENT_NAME            object 
 5   VALUE                     object 
 6   VALUE_STATE               int64  
 7   ACKNOWLEDGEMENT_REQUIRED  bool   
 8   SEVERITY                  int64  
 9   HIDDEN                    bool   
 10  THEME                     int64  
 11  EQUIPMENT_DATE            object 
 12  ACQUISITION_DATE          object 
 13  SCS_TIME                  object 
 14  FUNCTIONAL_CATEGORY       int64  
 15  GEOGRAPHICAL_CATEGORY     int64  
 16  ENVIRONMENT               object 
 17  USER1                     float64
 18  ASSET_ID_RAW              object 
 19  ASSET_DESCRIPTION         object 
 20  EVENT_D

In [14]:
# Inspect df
df_ae.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE
0,+,-1183235447,12362,0,:FRP:COMS:CCTS_0001:CAMS_0107,0,0,True,3,True,0,2021-01-24 03:06:40.343689984,2021-01-24 03:06:40.343689984,2021-01-24 03:06:40.343689984,40,10,OCCCMS,1.0,COM/FRP/B2/CAM07,FRP:107 SB TWC,Status,NORMAL,NaN,SUBLOCATION: SB TWC,Status,NaN,NaN,NaN,COM,CAM,2021-01-24 03:06:41.252000000,2021-01-24 03:06:41.252000000,2021-01-24 03:06:41.252604928
1,+,-1183235448,12363,0,:DBG:COMS:CCTS_0001:CAMS_0424,0,0,True,3,True,0,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,40,8,OCCCMS,1.0,COM/DBG/B2/CAM57,DBG:424 AFG 3,Status,NORMAL,NaN,AFG,Status,NaN,NaN,NaN,COM,CAM,2021-01-24 03:06:41.252000000,2021-01-24 03:06:41.252000000,2021-01-24 03:06:41.252604928
2,+,-1183235449,12364,0,:BNK:COMS:TELS_0001:PABS_0001,1,0,True,2,True,0,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,48,11,OCCCMS,1.0,COM/BNK/B1/PABX01,PABX,Fan 1 Status,NORMAL,NaN,PABX,Fan Status,NaN,NaN,NaN,COM,PABX,2021-01-24 03:06:41.252000000,2021-01-24 03:06:41.252000000,2021-01-24 03:06:41.252604928
3,+,-1183235450,4340,0,:HGN:COMS:PISS_0001:PDC__0001:PIDS_0011,9,1,True,2,True,0,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,44,16,OCCCMS,1.0,COM/HGN/1/PID11,Ent1:LD33,PID Status,FAN ALARM,NaN,Ent:LD,PID Status,NaN,NaN,NaN,COM,PID,2021-01-24 03:06:41.252000000,2021-01-24 03:06:41.252000000,2021-01-24 03:06:41.252604928
4,+,-1183235451,12365,0,:PGL:BMF:Supervision:LENV_SMS_,1,1,True,4,True,0,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,60,19,OCCCMS,1.0,PGL_LENV_SMS_,STN SMS Server - Environment PGLSMS,Environment 1 Status,STOP,NaN,STN SMS Server - Environment SMS,Environment Status,NaN,NaN,NaN,LENV,SMS,2021-01-24 03:06:41.252000000,2021-01-24 03:06:41.252000000,2021-01-24 03:06:41.252604928


### Append Alarm Tag Info

In [15]:
# Append in Alarm Tag (update 1.4)
df = df_ae.merge(df_al, "left", on=["ALARM_ID", "SCS_TIME", "ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW", "EVENT_DESCRIPTION", "EVENT_STATUS"])

# Set Null Alarm Tags as False
df.loc[df["isAlarm"].isnull(), "isAlarm"] = False

# Delete redundant variables
del df_al, df_ae

# Format time values to time format
df['DATETIME_SENT'] = pd.to_datetime(df['DATETIME_SENT'])
df['DATETIME_RECEIVED'] = pd.to_datetime(df['DATETIME_RECEIVED'])
df['EQUIPMENT_DATE'] = pd.to_datetime(df['EQUIPMENT_DATE'])
df['ACQUISITION_DATE'] = pd.to_datetime(df['ACQUISITION_DATE'])
df['SCS_TIME'] = pd.to_datetime(df['SCS_TIME'])
df['TIME_CODE'] = pd.to_datetime(df['TIME_CODE'])

# Print df summary stats
print(df.shape)
print(df.info())

(19604630, 34)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19604630 entries, 0 to 19604629
Data columns (total 34 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   ENTRY_CODE_SUFFIX         object        
 1   ENTRY_CODE                int64         
 2   ALARM_ID                  int64         
 3   USER_ID                   int64         
 4   EQUIPMENT_NAME            object        
 5   VALUE                     object        
 6   VALUE_STATE               int64         
 7   ACKNOWLEDGEMENT_REQUIRED  bool          
 8   SEVERITY                  int64         
 9   HIDDEN                    bool          
 10  THEME                     int64         
 11  EQUIPMENT_DATE            datetime64[ns]
 12  ACQUISITION_DATE          datetime64[ns]
 13  SCS_TIME                  datetime64[ns]
 14  FUNCTIONAL_CATEGORY       int64         
 15  GEOGRAPHICAL_CATEGORY     int64         
 16  ENVIRONMENT               object     

In [16]:
# Inspect df
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm
0,+,-1183235447,12362,0,:FRP:COMS:CCTS_0001:CAMS_0107,0,0,True,3,True,0,2021-01-24 03:06:40.343689984,2021-01-24 03:06:40.343689984,2021-01-24 03:06:40.343689984,40,10,OCCCMS,1.0,COM/FRP/B2/CAM07,FRP:107 SB TWC,Status,NORMAL,NaN,SUBLOCATION: SB TWC,Status,NaN,NaN,NaN,COM,CAM,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,False
1,+,-1183235448,12363,0,:DBG:COMS:CCTS_0001:CAMS_0424,0,0,True,3,True,0,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,40,8,OCCCMS,1.0,COM/DBG/B2/CAM57,DBG:424 AFG 3,Status,NORMAL,NaN,AFG,Status,NaN,NaN,NaN,COM,CAM,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,False
2,+,-1183235449,12364,0,:BNK:COMS:TELS_0001:PABS_0001,1,0,True,2,True,0,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,48,11,OCCCMS,1.0,COM/BNK/B1/PABX01,PABX,Fan 1 Status,NORMAL,NaN,PABX,Fan Status,NaN,NaN,NaN,COM,PABX,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,False
3,+,-1183235450,4340,0,:HGN:COMS:PISS_0001:PDC__0001:PIDS_0011,9,1,True,2,True,0,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,44,16,OCCCMS,1.0,COM/HGN/1/PID11,Ent1:LD33,PID Status,FAN ALARM,NaN,Ent:LD,PID Status,NaN,NaN,NaN,COM,PID,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,True
4,+,-1183235451,12365,0,:PGL:BMF:Supervision:LENV_SMS_,1,1,True,4,True,0,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,60,19,OCCCMS,1.0,PGL_LENV_SMS_,STN SMS Server - Environment PGLSMS,Environment 1 Status,STOP,NaN,STN SMS Server - Environment SMS,Environment Status,NaN,NaN,NaN,LENV,SMS,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,True


In [17]:
# Inspect df
df.loc[df["isAlarm"] == True].head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm
3,+,-1183235450,4340,0,:HGN:COMS:PISS_0001:PDC__0001:PIDS_0011,9,1,True,2,True,0,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,44,16,OCCCMS,1.0,COM/HGN/1/PID11,Ent1:LD33,PID Status,FAN ALARM,NaN,Ent:LD,PID Status,NaN,NaN,NaN,COM,PID,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,True
4,+,-1183235451,12365,0,:PGL:BMF:Supervision:LENV_SMS_,1,1,True,4,True,0,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,60,19,OCCCMS,1.0,PGL_LENV_SMS_,STN SMS Server - Environment PGLSMS,Environment 1 Status,STOP,NaN,STN SMS Server - Environment SMS,Environment Status,NaN,NaN,NaN,LENV,SMS,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,True
5,+,-1183235452,12366,0,:PGL:BMF:Supervision:LENV_SMS_,1,1,True,4,True,0,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,60,19,OCCCMS,1.0,PGL_LENV_SMS_,STN SMS Server - Environment PGLSMS,Environment 2 Status,STOP,NaN,STN SMS Server - Environment SMS,Environment Status,NaN,NaN,NaN,LENV,SMS,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,True
6,+,-1183235453,12367,0,:SKG:BMF:Supervision:LENV_SMS_,1,1,True,4,True,0,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.230729216,60,18,OCCCMS,1.0,SKG_LENV_SMS_,STN SMS Server - Environment SKGSMS,Environment 1 Status,STOP,NaN,STN SMS Server - Environment SUBLOCATIONSMS,Environment Status,NaN,NaN,NaN,LENV,SMS,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,True
7,+,-1183235454,12368,0,:SKG:BMF:Supervision:LENV_SMS_,1,1,True,4,True,0,2021-01-24 03:06:41.230729216,2021-01-24 03:06:41.230729216,2021-01-24 03:06:41.230729216,60,18,OCCCMS,1.0,SKG_LENV_SMS_,STN SMS Server - Environment SKGSMS,Environment 2 Status,STOP,NaN,STN SMS Server - Environment SUBLOCATIONSMS,Environment Status,NaN,NaN,NaN,LENV,SMS,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,True


### Tag Toggle Alarm/Events

#### Reindex Data By Time

In [18]:
# Create copy of time sequence field (SCS_TIME) as a proto index column
df["datetimeIndex"] = df["SCS_TIME"].copy()

# Set Date-Time to be the index
df = df.set_index(df["datetimeIndex"])

# Drop proto index column
del df["datetimeIndex"]

# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm
datetimeIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-24 03:06:40.343689984,+,-1183235447,12362,0,:FRP:COMS:CCTS_0001:CAMS_0107,0,0,True,3,True,0,2021-01-24 03:06:40.343689984,2021-01-24 03:06:40.343689984,2021-01-24 03:06:40.343689984,40,10,OCCCMS,1.0,COM/FRP/B2/CAM07,FRP:107 SB TWC,Status,NORMAL,NaN,SUBLOCATION: SB TWC,Status,NaN,NaN,NaN,COM,CAM,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,False
2021-01-24 03:06:40.528445952,+,-1183235448,12363,0,:DBG:COMS:CCTS_0001:CAMS_0424,0,0,True,3,True,0,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,40,8,OCCCMS,1.0,COM/DBG/B2/CAM57,DBG:424 AFG 3,Status,NORMAL,NaN,AFG,Status,NaN,NaN,NaN,COM,CAM,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,False
2021-01-24 03:06:40.528445952,+,-1183235449,12364,0,:BNK:COMS:TELS_0001:PABS_0001,1,0,True,2,True,0,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,48,11,OCCCMS,1.0,COM/BNK/B1/PABX01,PABX,Fan 1 Status,NORMAL,NaN,PABX,Fan Status,NaN,NaN,NaN,COM,PABX,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,False
2021-01-24 03:06:40.528445952,+,-1183235450,4340,0,:HGN:COMS:PISS_0001:PDC__0001:PIDS_0011,9,1,True,2,True,0,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,2021-01-24 03:06:40.528445952,44,16,OCCCMS,1.0,COM/HGN/1/PID11,Ent1:LD33,PID Status,FAN ALARM,NaN,Ent:LD,PID Status,NaN,NaN,NaN,COM,PID,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,True
2021-01-24 03:06:41.324359936,+,-1183235451,12365,0,:PGL:BMF:Supervision:LENV_SMS_,1,1,True,4,True,0,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,2021-01-24 03:06:41.324359936,60,19,OCCCMS,1.0,PGL_LENV_SMS_,STN SMS Server - Environment PGLSMS,Environment 1 Status,STOP,NaN,STN SMS Server - Environment SMS,Environment Status,NaN,NaN,NaN,LENV,SMS,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252,2021-01-24 03:06:41.252604928,True


In [19]:
# Sort Data By ASSETNAME and DT (DEFAULT)
# UPDATE: 
# Dropped ALARM_ID as repeat and toggling events can appear in different ALARM_ID for the same asset
# In Close Succession
df = df.sort_index().sort_values(["ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW", "SCS_TIME"])

# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm
datetimeIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-04 09:14:51.804434176,+,-1172822874,4327307,0,:BGK:BMF:CBN__0001:ACMC_0003,0,0,True,2,True,0,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502918912,False
2021-01-04 09:28:37.320538880,+,-1172827020,4331149,0,:BGK:BMF:CBN__0001:ACMC_0003,1,0,True,2,True,0,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502679040,False
2021-01-07 13:04:31.937200128,+,-1174122650,5490800,0,:BGK:BMF:CBN__0001:ACMC_0004,0,0,True,2,True,0,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505182976,False
2021-01-11 10:42:39.180665088,+,-1177593274,416199,0,:BGK:BMF:CBN__0001:ACMC_0004,1,0,True,2,True,0,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504775936,False
2021-01-12 04:23:55.872451072,+,-1177850988,653999,0,:BGK:BMF:CBN__0001:SDHB_0001,1,1,True,4,True,0,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/SDH02,Synchronous Digital Hierarchy A,Major Alarm,ALARM,NaN,Synchronous Digital Hierarchy A,Major Alarm,NaN,NaN,NaN,NEL,SDH,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505615104,True


#### Nuisance Alarm Suppression

In [20]:
# Concat Event Description + Status values
# Note: This definition also treats event changes due to user acknowledgement as potential repeat alarms
df["eventValue_N"] = df["EVENT_DESCRIPTION"] + " | " + df["EVENT_STATUS"]

# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm,eventValue_N
datetimeIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-04 09:14:51.804434176,+,-1172822874,4327307,0,:BGK:BMF:CBN__0001:ACMC_0003,0,0,True,2,True,0,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502918912,False,Minor Alarm | NORMAL
2021-01-04 09:28:37.320538880,+,-1172827020,4331149,0,:BGK:BMF:CBN__0001:ACMC_0003,1,0,True,2,True,0,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502679040,False,Minor Alarm | ALARM
2021-01-07 13:04:31.937200128,+,-1174122650,5490800,0,:BGK:BMF:CBN__0001:ACMC_0004,0,0,True,2,True,0,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505182976,False,Minor Alarm | NORMAL
2021-01-11 10:42:39.180665088,+,-1177593274,416199,0,:BGK:BMF:CBN__0001:ACMC_0004,1,0,True,2,True,0,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504775936,False,Minor Alarm | ALARM
2021-01-12 04:23:55.872451072,+,-1177850988,653999,0,:BGK:BMF:CBN__0001:SDHB_0001,1,1,True,4,True,0,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/SDH02,Synchronous Digital Hierarchy A,Major Alarm,ALARM,NaN,Synchronous Digital Hierarchy A,Major Alarm,NaN,NaN,NaN,NEL,SDH,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505615104,True,Major Alarm | ALARM


In [21]:
# UPDATE: 
# Dropped ALARM_ID as repeat and toggling events can appear in different ALARM_ID for the same asset
# In Close Succession

# Past 5 values for the same ["ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW", "ALARM_ID"] within in a 1 min window
df["eventValue_N-1"] = df.groupby(["ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW"])["eventValue_N"].shift(1)
df["eventValue_N-2"] = df.groupby(["ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW"])["eventValue_N"].shift(2)
df["eventValue_N-3"] = df.groupby(["ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW"])["eventValue_N"].shift(3)
df["eventValue_N-4"] = df.groupby(["ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW"])["eventValue_N"].shift(4)
df["eventValue_N-5"] = df.groupby(["ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW"])["eventValue_N"].shift(5)

# Get Timestamps of N-1 and N-5
df["SCS_TIME_N-1"] = df.groupby(["ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW"])["SCS_TIME"].shift(1)
df["SCS_TIME_N-3"] = df.groupby(["ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW"])["SCS_TIME"].shift(3)
df["SCS_TIME_N-5"] = df.groupby(["ENVIRONMENT", "EQUIPMENT_NAME", "ASSET_ID_RAW"])["SCS_TIME"].shift(5)

# Inspect data sample
#df.head()
df.loc[~df["eventValue_N-1"].isnull()].head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm,eventValue_N,eventValue_N-1,eventValue_N-2,eventValue_N-3,eventValue_N-4,eventValue_N-5,SCS_TIME_N-1,SCS_TIME_N-3,SCS_TIME_N-5
datetimeIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-04 09:28:37.320538880,+,-1172827020,4331149,0,:BGK:BMF:CBN__0001:ACMC_0003,1,0,True,2,True,0,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502679040,False,Minor Alarm | ALARM,Minor Alarm | NORMAL,NaN,NaN,NaN,NaN,2021-01-04 09:14:51.804434176,NaT,NaT
2021-01-11 10:42:39.180665088,+,-1177593274,416199,0,:BGK:BMF:CBN__0001:ACMC_0004,1,0,True,2,True,0,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504775936,False,Minor Alarm | ALARM,Minor Alarm | NORMAL,NaN,NaN,NaN,NaN,2021-01-07 13:04:31.937200128,NaT,NaT
2021-01-12 04:26:50.984546816,+,-1177851724,653999,0,:BGK:BMF:CBN__0001:SDHB_0001,0,0,True,2,True,0,2021-01-12 04:26:50.984546816,2021-01-12 04:26:50.984546816,2021-01-12 04:26:50.984546816,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/SDH02,Synchronous Digital Hierarchy A,Major Alarm,NORMAL,NaN,Synchronous Digital Hierarchy A,Major Alarm,NaN,NaN,NaN,NEL,SDH,2021-01-12 04:26:51.504,2021-01-12 04:26:51.504,2021-01-12 04:26:51.504555008,True,Major Alarm | NORMAL,Major Alarm | ALARM,NaN,NaN,NaN,NaN,2021-01-12 04:23:55.872451072,NaT,NaT
2021-01-12 04:30:48.164097024,+,-1177852662,653999,0,:BGK:BMF:CBN__0001:SDHB_0001,0,0,True,2,True,0,2021-01-12 04:26:50.984546816,2021-01-12 04:26:50.984546816,2021-01-12 04:30:48.164097024,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/SDH02,Synchronous Digital Hierarchy A,Major Alarm,NORMAL,HLC,Synchronous Digital Hierarchy A,Major Alarm,NaN,NaN,NaN,NEL,SDH,2021-01-12 04:30:48.508,2021-01-12 04:30:48.508,2021-01-12 04:30:48.508014848,False,Major Alarm | NORMAL,Major Alarm | NORMAL,Major Alarm | ALARM,NaN,NaN,NaN,2021-01-12 04:26:50.984546816,NaT,NaT
2020-12-24 18:58:30.812616192,+,-1169247110,668836,0,:BGK:BMF:CBN__0001:SDHB_0002,0,0,True,2,True,0,2020-12-24 18:58:30.812616192,2020-12-24 18:58:30.812616192,2020-12-24 18:58:30.812616192,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/SDH04,Synchronous Digital Hierarchy B,Major Alarm,NORMAL,NaN,Synchronous Digital Hierarchy B,Major Alarm,NaN,NaN,NaN,NEL,SDH,2020-12-24 18:58:31.502,2020-12-24 18:58:31.502,2020-12-24 18:58:31.502824960,True,Major Alarm | NORMAL,Major Alarm | ALARM,NaN,NaN,NaN,NaN,2020-12-24 18:58:21.236509952,NaT,NaT


In [22]:
# Reset index to serial number
df = df.reset_index()

# Drop former index column
df = df.drop(columns = ["datetimeIndex"])

# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm,eventValue_N,eventValue_N-1,eventValue_N-2,eventValue_N-3,eventValue_N-4,eventValue_N-5,SCS_TIME_N-1,SCS_TIME_N-3,SCS_TIME_N-5
0,+,-1172822874,4327307,0,:BGK:BMF:CBN__0001:ACMC_0003,0,0,True,2,True,0,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502918912,False,Minor Alarm | NORMAL,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
1,+,-1172827020,4331149,0,:BGK:BMF:CBN__0001:ACMC_0003,1,0,True,2,True,0,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502679040,False,Minor Alarm | ALARM,Minor Alarm | NORMAL,NaN,NaN,NaN,NaN,2021-01-04 09:14:51.804434176,NaT,NaT
2,+,-1174122650,5490800,0,:BGK:BMF:CBN__0001:ACMC_0004,0,0,True,2,True,0,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505182976,False,Minor Alarm | NORMAL,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
3,+,-1177593274,416199,0,:BGK:BMF:CBN__0001:ACMC_0004,1,0,True,2,True,0,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504775936,False,Minor Alarm | ALARM,Minor Alarm | NORMAL,NaN,NaN,NaN,NaN,2021-01-07 13:04:31.937200128,NaT,NaT
4,+,-1177850988,653999,0,:BGK:BMF:CBN__0001:SDHB_0001,1,1,True,4,True,0,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/SDH02,Synchronous Digital Hierarchy A,Major Alarm,ALARM,NaN,Synchronous Digital Hierarchy A,Major Alarm,NaN,NaN,NaN,NEL,SDH,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505615104,True,Major Alarm | ALARM,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT


In [23]:
# Initialise Field with an Empty Column
#df.reindex(columns = df.columns.tolist() + ["NuisanceAlarm"])
df["NuisanceAlarm"] = False
df["RepeatAlarm"] = False
df["AltAlarm2"] = False
df["AltAlarm3"] = False

# Check for consecutive repeat alarms
df.loc[(~df["eventValue_N-1"].isnull()) & # ensure that there is a value to compare to
       (df["SCS_TIME_N-1"] <= (df["SCS_TIME"] + pd.Timedelta(seconds=60))) & # check if timeframe is within 60s
       (df["eventValue_N"] == df["eventValue_N-1"]) # compare values for repeat values
      , "NuisanceAlarm"] = True

# Check for toggle alarms (3 Alternating Pairs)
df.loc[(~df["eventValue_N-3"].isnull()) & # ensure that there is a value to compare to
       (df["SCS_TIME_N-3"] <= (df["SCS_TIME"] + pd.Timedelta(seconds=60))) & # check if timeframe is within 60s
       (df["eventValue_N"] != df["eventValue_N-1"]) & # ensure the eventValue_N (A) and eventValue_N-1 (B) are different
       (df["eventValue_N"] == df["eventValue_N-2"]) & # check for alternating pattern A1
       (df["eventValue_N-1"] == df["eventValue_N-3"]) # check for alternating pattern B1
      , "NuisanceAlarm"] = True

# Additional Code for Testing

# Check for consecutive repeat alarms
df.loc[(~df["eventValue_N-1"].isnull()) & # ensure that there is a value to compare to
       (df["SCS_TIME_N-1"] <= (df["SCS_TIME"] + pd.Timedelta(seconds=60))) & # check if timeframe is within 60s
       (df["eventValue_N"] == df["eventValue_N-1"]) # compare values for repeat values
      , "RepeatAlarm"] = True

# Check for toggle alarms (3 Alternating Pairs)
df.loc[(~df["eventValue_N-3"].isnull()) & # ensure that there is a value to compare to
       (df["SCS_TIME_N-3"] <= (df["SCS_TIME"] + pd.Timedelta(seconds=60))) & # check if timeframe is within 60s
       (df["eventValue_N"] != df["eventValue_N-1"]) & # ensure the eventValue_N (A) and eventValue_N-1 (B) are different
       (df["eventValue_N"] == df["eventValue_N-2"]) & # check for alternating pattern A1
       (df["eventValue_N-1"] == df["eventValue_N-3"]) # check for alternating pattern B1
      , "AltAlarm2"] = True

# Check for toggle alarms (3 Alternating Pairs)
df.loc[(~df["eventValue_N-5"].isnull()) & # ensure that there is a value to compare to
       (df["SCS_TIME_N-5"] <= (df["SCS_TIME"] + pd.Timedelta(seconds=60))) & # check if timeframe is within 60s
       (df["eventValue_N"] != df["eventValue_N-1"]) & # ensure the eventValue_N (A) and eventValue_N-1 (B) are different
       (df["eventValue_N"] == df["eventValue_N-2"]) & # check for alternating pattern A1
       (df["eventValue_N-1"] == df["eventValue_N-3"]) & # check for alternating pattern B1
       (df["eventValue_N"] == df["eventValue_N-4"]) & # check for alternating pattern A2
       (df["eventValue_N-1"] == df["eventValue_N-5"]) # check for alternating pattern B2
      , "AltAlarm3"] = True

# Inspect data sample
df.head()
#df.loc[df["NuisanceAlarm"] == True].head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm,eventValue_N,eventValue_N-1,eventValue_N-2,eventValue_N-3,eventValue_N-4,eventValue_N-5,SCS_TIME_N-1,SCS_TIME_N-3,SCS_TIME_N-5,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3
0,+,-1172822874,4327307,0,:BGK:BMF:CBN__0001:ACMC_0003,0,0,True,2,True,0,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502918912,False,Minor Alarm | NORMAL,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,False,False,False,False
1,+,-1172827020,4331149,0,:BGK:BMF:CBN__0001:ACMC_0003,1,0,True,2,True,0,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502679040,False,Minor Alarm | ALARM,Minor Alarm | NORMAL,NaN,NaN,NaN,NaN,2021-01-04 09:14:51.804434176,NaT,NaT,False,False,False,False
2,+,-1174122650,5490800,0,:BGK:BMF:CBN__0001:ACMC_0004,0,0,True,2,True,0,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505182976,False,Minor Alarm | NORMAL,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,False,False,False,False
3,+,-1177593274,416199,0,:BGK:BMF:CBN__0001:ACMC_0004,1,0,True,2,True,0,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504775936,False,Minor Alarm | ALARM,Minor Alarm | NORMAL,NaN,NaN,NaN,NaN,2021-01-07 13:04:31.937200128,NaT,NaT,False,False,False,False
4,+,-1177850988,653999,0,:BGK:BMF:CBN__0001:SDHB_0001,1,1,True,4,True,0,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/SDH02,Synchronous Digital Hierarchy A,Major Alarm,ALARM,NaN,Synchronous Digital Hierarchy A,Major Alarm,NaN,NaN,NaN,NEL,SDH,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505615104,True,Major Alarm | ALARM,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,False,False,False,False


In [24]:
# Delete helper columns
df = df.drop(columns=["eventValue_N", 
                      "eventValue_N-1",
                      "eventValue_N-2",
                      "eventValue_N-3",
                      "eventValue_N-4",
                      "eventValue_N-5",
                      "SCS_TIME_N-1",
                      "SCS_TIME_N-3",
                      "SCS_TIME_N-5"])

# Inspect data sample
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3
0,+,-1172822874,4327307,0,:BGK:BMF:CBN__0001:ACMC_0003,0,0,True,2,True,0,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502918912,False,False,False,False,False
1,+,-1172827020,4331149,0,:BGK:BMF:CBN__0001:ACMC_0003,1,0,True,2,True,0,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502679040,False,False,False,False,False
2,+,-1174122650,5490800,0,:BGK:BMF:CBN__0001:ACMC_0004,0,0,True,2,True,0,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505182976,False,False,False,False,False
3,+,-1177593274,416199,0,:BGK:BMF:CBN__0001:ACMC_0004,1,0,True,2,True,0,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504775936,False,False,False,False,False
4,+,-1177850988,653999,0,:BGK:BMF:CBN__0001:SDHB_0001,1,1,True,4,True,0,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/SDH02,Synchronous Digital Hierarchy A,Major Alarm,ALARM,NaN,Synchronous Digital Hierarchy A,Major Alarm,NaN,NaN,NaN,NEL,SDH,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505615104,True,False,False,False,False


In [25]:
# Inspect data structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19604630 entries, 0 to 19604629
Data columns (total 38 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   ENTRY_CODE_SUFFIX         object        
 1   ENTRY_CODE                int64         
 2   ALARM_ID                  int64         
 3   USER_ID                   int64         
 4   EQUIPMENT_NAME            object        
 5   VALUE                     object        
 6   VALUE_STATE               int64         
 7   ACKNOWLEDGEMENT_REQUIRED  bool          
 8   SEVERITY                  int64         
 9   HIDDEN                    bool          
 10  THEME                     int64         
 11  EQUIPMENT_DATE            datetime64[ns]
 12  ACQUISITION_DATE          datetime64[ns]
 13  SCS_TIME                  datetime64[ns]
 14  FUNCTIONAL_CATEGORY       int64         
 15  GEOGRAPHICAL_CATEGORY     int64         
 16  ENVIRONMENT               object        
 17  USER1 

In [26]:
print("Nuisance Alarms Stats")
print()
print("Total: " + str(df["RepeatAlarm"].count()))
print("Total Alarms: " + str(df.loc[df["isAlarm"] == True, "RepeatAlarm"].count()))
print("Total Alarms against global total %: " + str(df.loc[df["isAlarm"] == True, "RepeatAlarm"].count() / df["RepeatAlarm"].count() * 100))
print()
print("Repeat Events: " + str(df.loc[df["RepeatAlarm"] == True, "RepeatAlarm"].count()))
print("Repeat Events against global total %: " + str(df.loc[df["RepeatAlarm"] == True, "RepeatAlarm"].count() / df["RepeatAlarm"].count() * 100))
print("Repeat Events which are alarms against global total %: " + str(df.loc[(df["RepeatAlarm"] == True) & (df["isAlarm"] == True), "RepeatAlarm"].count() / df["RepeatAlarm"].count() * 100))
print()
print("AltAlarm2 Events: " + str(df.loc[df["AltAlarm2"] == True, "RepeatAlarm"].count()))
print("AltAlarm2 Events against global total %: " + str(df.loc[df["AltAlarm2"] == True, "RepeatAlarm"].count() / df["RepeatAlarm"].count() * 100))
print("AltAlarm2 Events which are alarms against global total %: " + str(df.loc[(df["AltAlarm2"] == True) & (df["isAlarm"] == True), "RepeatAlarm"].count() / df["RepeatAlarm"].count() * 100))
print()
print("AltAlarm3 Events: " + str(df.loc[df["AltAlarm3"] == True, "RepeatAlarm"].count()))
print("AltAlarm3 Events against global total %: " + str(df.loc[df["AltAlarm3"] == True, "RepeatAlarm"].count() / df["RepeatAlarm"].count() * 100))
print("AltAlarm3 Events which are alarms against global total %: " + str(df.loc[(df["AltAlarm3"] == True) & (df["isAlarm"] == True), "RepeatAlarm"].count() / df["RepeatAlarm"].count() * 100))


Nuisance Alarms Stats

Total: 19604630
Total Alarms: 826833
Total Alarms against global total %: 4.217539428186097

Repeat Events: 1876958
Repeat Events against global total %: 9.574054700343746
Repeat Events which are alarms against global total %: 0.3267289410715734

AltAlarm2 Events: 14660818
AltAlarm2 Events against global total %: 74.78242639621355
AltAlarm2 Events which are alarms against global total %: 2.9934714401649

AltAlarm3 Events: 13168720
AltAlarm3 Events against global total %: 67.17147939032769
AltAlarm3 Events which are alarms against global total %: 2.645069047464808


### Sort Data by Time

In [27]:
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3
0,+,-1172822874,4327307,0,:BGK:BMF:CBN__0001:ACMC_0003,0,0,True,2,True,0,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,2021-01-04 09:14:51.804434176,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502,2021-01-04 09:14:52.502918912,False,False,False,False,False
1,+,-1172827020,4331149,0,:BGK:BMF:CBN__0001:ACMC_0003,1,0,True,2,True,0,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,2021-01-04 09:28:37.320538880,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/ACM05,OPT-E1 Access Multiplexer A,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer A,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502,2021-01-04 09:28:37.502679040,False,False,False,False,False
2,+,-1174122650,5490800,0,:BGK:BMF:CBN__0001:ACMC_0004,0,0,True,2,True,0,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,2021-01-07 13:04:31.937200128,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,NORMAL,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505,2021-01-07 13:04:31.505182976,False,False,False,False,False
3,+,-1177593274,416199,0,:BGK:BMF:CBN__0001:ACMC_0004,1,0,True,2,True,0,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,2021-01-11 10:42:39.180665088,51,17,OCCCMS,1.0,NEL/COM/BGK/B2/ACM06,OPT-E1 Access Multiplexer B,Minor Alarm,ALARM,NaN,OPT-E Access Multiplexer B,Minor Alarm,NaN,NaN,NaN,NEL,ACM,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504,2021-01-11 10:42:39.504775936,False,False,False,False,False
4,+,-1177850988,653999,0,:BGK:BMF:CBN__0001:SDHB_0001,1,1,True,4,True,0,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,2021-01-12 04:23:55.872451072,51,17,OCCCMS,1.0,NEL/COM/BGK/B1/SDH02,Synchronous Digital Hierarchy A,Major Alarm,ALARM,NaN,Synchronous Digital Hierarchy A,Major Alarm,NaN,NaN,NaN,NEL,SDH,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505,2021-01-12 04:23:56.505615104,True,False,False,False,False


In [28]:
df.tail()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3
19604625,+,-1181505319,-29522,0,NaN,0.000000e+00,0,True,3,True,-1,2021-01-19 10:45:20.501585920,2021-01-19 10:45:20.501585920,2021-01-19 10:45:20.501585920,1,1,OCCCMS,NaN,TUNNEL_LIGHT_ON,Tunnel LTG,Turn ON NB and SB and CS Tunnel Light,TERMINATED,RBN,Tunnel LTG,Turn ON NB and SB and CS Tunnel Light,NaN,NaN,NaN,LIGHT,ON,2021-01-19 10:45:21.003,2021-01-19 10:45:21.003,2021-01-19 10:45:21.378000128,False,False,False,False,False
19604626,+,-1183240786,-137,0,NaN,0.000000e+00,0,True,3,True,-1,2021-01-24 03:36:51.309040128,2021-01-24 03:36:51.309040128,2021-01-24 03:36:51.309040128,1,1,OCCCMS,NaN,TUNNEL_LIGHT_ON,Tunnel LTG,Turn ON NB and SB and CS Tunnel Light,REQUESTED,RBN,Tunnel LTG,Turn ON NB and SB and CS Tunnel Light,NaN,NaN,NaN,LIGHT,ON,2021-01-24 03:36:52.259,2021-01-24 03:36:52.259,2021-01-24 03:36:52.259663104,False,False,False,False,False
19604627,+,-1183241869,-140,0,NaN,0.000000e+00,0,True,3,True,-1,2021-01-24 03:40:54.255806976,2021-01-24 03:40:54.255806976,2021-01-24 03:40:54.255806976,1,1,OCCCMS,NaN,TUNNEL_LIGHT_ON,Tunnel LTG,Turn ON NB and SB and CS Tunnel Light,TERMINATED,RBN,Tunnel LTG,Turn ON NB and SB and CS Tunnel Light,NaN,NaN,NaN,LIGHT,ON,2021-01-24 03:40:54.260,2021-01-24 03:40:54.260,2021-01-24 03:40:54.260729088,False,False,False,False,False
19604628,+,-1185621009,-136744,0,NaN,0.000000e+00,0,True,3,True,-1,2021-01-28 10:53:27.301433088,2021-01-28 10:53:27.301433088,2021-01-28 10:53:27.301433088,1,1,OCCCMS,NaN,TUNNEL_LIGHT_ON,Tunnel LTG,Turn ON NB and SB and CS Tunnel Light,REQUESTED,RBN,Tunnel LTG,Turn ON NB and SB and CS Tunnel Light,NaN,NaN,NaN,LIGHT,ON,2021-01-28 10:53:28.256,2021-01-28 10:53:28.256,2021-01-28 10:53:28.256849920,False,False,False,False,False
19604629,+,-1185621786,-136899,0,NaN,0.000000e+00,0,True,3,True,-1,2021-01-28 10:55:08.751492096,2021-01-28 10:55:08.751492096,2021-01-28 10:55:08.751492096,1,1,OCCCMS,NaN,TUNNEL_LIGHT_ON,Tunnel LTG,Turn ON NB and SB and CS Tunnel Light,TERMINATED,RBN,Tunnel LTG,Turn ON NB and SB and CS Tunnel Light,NaN,NaN,NaN,LIGHT,ON,2021-01-28 10:55:09.253,2021-01-28 10:55:09.253,2021-01-28 10:55:09.253475072,False,False,False,False,False


In [29]:
# Sort data by time for ease of export in chronological order (TEST)
#df.sort_values(["SCS_TIME"], ignore_index=True) 

In [30]:
# Sort data by time for ease of export in chronological order
df = df.sort_values(["SCS_TIME"], ignore_index=True) 

In [31]:
df.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3
0,+,-1168181832,12654,0,:PTP:BMF:Supervision:LENV_GWS_,1.0,1,True,4,True,0,2020-12-23 01:13:54.605474048,2020-12-23 01:13:54.605474048,2020-12-23 01:13:54.605474048,60,12,OCCCMS,1.0,SCS/PTP/B1/GWS01,SCR GWS 1,Environment Status,STOPPED,NaN,SCR GWS,Environment Status,NaN,NaN,NaN,SCS,GWS,2020-12-23 01:13:55.501,2020-12-23 01:13:55.501,2020-12-23 01:13:55.501458944,False,False,False,False,False
1,+,-1168181816,12638,0,:OCC:BMF:Supervision:LENV_GWS2,1.0,1,True,4,True,0,2020-12-23 01:13:54.605474048,2020-12-23 01:13:54.605474048,2020-12-23 01:13:54.605474048,60,1,OCCCMS,1.0,SCS/NED/1212/GWS02,OCC GWS 2,Environment Status,STOPPED,NaN,GWS,Environment Status,NaN,NaN,NaN,SCS,GWS,2020-12-23 01:13:55.501,2020-12-23 01:13:55.501,2020-12-23 01:13:55.501458944,False,False,False,False,False
2,+,-1168181804,12626,0,:BGK:BMF:Supervision:LENV_SMS_,1.0,1,True,4,True,0,2020-12-23 01:13:54.605474048,2020-12-23 01:13:54.605474048,2020-12-23 01:13:54.605474048,60,17,OCCCMS,1.0,BGK_LENV_SMS_,STN SMS Server - Environment BGKSMS,Environment 2 Status,STOP,NaN,STN SMS Server - Environment SUBLOCATIONSMS,Environment Status,NaN,NaN,NaN,LENV,SMS,2020-12-23 01:13:55.501,2020-12-23 01:13:55.501,2020-12-23 01:13:55.501458944,False,False,False,False,False
3,+,-1168181817,12639,0,:OCC:BMF:Supervision:LENV_GWS3,1.0,1,True,4,True,0,2020-12-23 01:13:54.605474048,2020-12-23 01:13:54.605474048,2020-12-23 01:13:54.605474048,60,1,OCCCMS,1.0,SCS/NED/1212/GWS03,OCC GWS 3,Environment Status,STOPPED,NaN,GWS,Environment Status,NaN,NaN,NaN,SCS,GWS,2020-12-23 01:13:55.501,2020-12-23 01:13:55.501,2020-12-23 01:13:55.501458944,False,False,False,False,False
4,+,-1168181803,12625,0,:BGK:BMF:Supervision:LENV_SMS_,1.0,1,True,4,True,0,2020-12-23 01:13:54.605474048,2020-12-23 01:13:54.605474048,2020-12-23 01:13:54.605474048,60,17,OCCCMS,1.0,BGK_LENV_SMS_,STN SMS Server - Environment BGKSMS,Environment 1 Status,STOP,NaN,STN SMS Server - Environment SUBLOCATIONSMS,Environment Status,NaN,NaN,NaN,LENV,SMS,2020-12-23 01:13:55.501,2020-12-23 01:13:55.501,2020-12-23 01:13:55.501458944,False,False,False,False,False


In [32]:
df.tail()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,CarID,ServiceID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3
19604625,+,-1187289723,570957,0,:CNT:COMS:CCTS_0001:CAMS_0219,0,0,True,3,True,0,2021-02-01 02:32:30.938765056,2021-02-01 02:32:30.938765056,2021-02-01 02:32:30.938765056,40,6,OCCCMS,1.0,COM/CNT/B1/CAM41,CNT:219 AFG 12,Status,NORMAL,NaN,SUBLOCATION: AFG,Status,NaN,NaN,NaN,COM,CAM,2021-02-01 02:32:31.002,2021-02-01 02:32:31.002,2021-02-01 02:32:31.298599936,False,True,False,True,True
19604626,+,-1187289724,570958,0,:HBF:COMS:CCTS_0001:CAMS_0205,1,0,True,3,True,0,2021-02-01 02:32:30.938765056,2021-02-01 02:32:30.938765056,2021-02-01 02:32:30.938765056,40,4,OCCCMS,1.0,COM/HBF/B1/CAM17,HBF:205 C ES02,Status,FAILURE,NaN,C ES,Status,NaN,NaN,NaN,COM,CAM,2021-02-01 02:32:31.002,2021-02-01 02:32:31.002,2021-02-01 02:32:31.298599936,False,False,False,False,False
19604627,+,-1187289725,570959,0,:BNK:COMS:TELS_0001:PABS_0001,1,0,True,2,True,0,2021-02-01 02:32:31.190761216,2021-02-01 02:32:31.190761216,2021-02-01 02:32:31.190761216,48,11,OCCCMS,1.0,COM/BNK/B1/PABX01,PABX,Fan 1 Status,NORMAL,NaN,PABX,Fan Status,NaN,NaN,NaN,COM,PABX,2021-02-01 02:32:32.002,2021-02-01 02:32:32.002,2021-02-01 02:32:32.214499840,False,True,False,True,False
19604628,+,-1187289726,570960,0,:HBF:COMS:CCTS_0001:CAMS_0205,0,0,True,3,True,0,2021-02-01 02:32:31.442743040,2021-02-01 02:32:31.442743040,2021-02-01 02:32:31.442743040,40,4,OCCCMS,1.0,COM/HBF/B1/CAM17,HBF:205 C ES02,Status,NORMAL,NaN,C ES,Status,NaN,NaN,NaN,COM,CAM,2021-02-01 02:32:32.002,2021-02-01 02:32:32.002,2021-02-01 02:32:32.214499840,False,True,False,True,False
19604629,+,-1187289727,570961,0,:DBG:COMS:CCTS_0001:CAMS_0424,0,0,True,3,True,0,2021-02-01 02:32:31.946716928,2021-02-01 02:32:31.946716928,2021-02-01 02:32:31.946716928,40,8,OCCCMS,1.0,COM/DBG/B2/CAM57,DBG:424 AFG 3,Status,NORMAL,NaN,SUBLOCATION: AFG,Status,NaN,NaN,NaN,COM,CAM,2021-02-01 02:32:32.002,2021-02-01 02:32:32.002,2021-02-01 02:32:32.214499840,False,True,True,False,False


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19604630 entries, 0 to 19604629
Data columns (total 38 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   ENTRY_CODE_SUFFIX         object        
 1   ENTRY_CODE                int64         
 2   ALARM_ID                  int64         
 3   USER_ID                   int64         
 4   EQUIPMENT_NAME            object        
 5   VALUE                     object        
 6   VALUE_STATE               int64         
 7   ACKNOWLEDGEMENT_REQUIRED  bool          
 8   SEVERITY                  int64         
 9   HIDDEN                    bool          
 10  THEME                     int64         
 11  EQUIPMENT_DATE            datetime64[ns]
 12  ACQUISITION_DATE          datetime64[ns]
 13  SCS_TIME                  datetime64[ns]
 14  FUNCTIONAL_CATEGORY       int64         
 15  GEOGRAPHICAL_CATEGORY     int64         
 16  ENVIRONMENT               object        
 17  USER1 

## Export File

In [34]:
# Check current directory
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [35]:
# Define Save Location
saveLoc = '\\taggedOutput\\'
#os.chdir(cwd + alarmLoc)
os.chdir(cwd + saveLoc)
# Check directory location
print(os.getcwd())

C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\taggedOutput


In [36]:
# Define File Save Parameters
FileName = "CMS"
Run = "-B0001"
# True for single file output; # False for multiple file output; "both" for both Single & Multiple File Output
singleSave = "both"

# Get length of dataframe
df_len = len(df)
# Inspect data
print(df_len)

# Define Size of Partitioned Dataframes
partionSize = 500000

# Define Number of Partitions (Always Round Up to Nearest Interger)
if (df_len == partionSize): 
    partitions = 1
else:
    partitions = df_len // partionSize + 1

# Inspect data
print(partitions)


19604630
40


In [37]:
# Export file based on above settings
if singleSave == True:
    fileNameN = FileName + Run + '-alarmsTagged' + '.csv'
    df.to_csv("main/" + fileNameN, index=False)
    print(fileNameN + " SAVED")
elif singleSave == "both":
    # Single File Save
    fileNameN = FileName + Run + '-alarmsTagged' + '.csv'
    df.to_csv("main/" + fileNameN, index=False)
    print(fileNameN + " SAVED")
    
    # Split Dataframe into batches of 500K rows
    for counter in range(partitions):
        startPoint = counter * partionSize
        df_subset = df.iloc[startPoint : (startPoint + partionSize)]
        #print(df_subset.info())

        # Save File
        fileNameN = "Subset File Ver/" + FileName + Run + '-' + str(counter).zfill(3) + '-alarmsTagged' + '.csv'
        df_subset.to_csv(fileNameN, index=False)
        print(fileNameN + " SAVED")
else:
    # Split Dataframe into batches of 500K rows
    for counter in range(partitions):
        startPoint = counter * partionSize
        df_subset = df.iloc[startPoint : (startPoint + partionSize)]
        #print(df_subset.info())

        # Save File
        fileNameN = "Subset File Ver/" + FileName + Run + '-' + str(counter).zfill(3) + '-alarmsTagged' + '.csv'
        df_subset.to_csv(fileNameN, index=False)
        print(fileNameN + " SAVED")
        
# Ring Beeper When Complete
import winsound
import time

duration1 = 400  # milliseconds
freq1 = 400  # Hz
duration2 = 600  # milliseconds
freq2 = 300  # Hz
repeatCount = 7
for n in range(repeatCount):
    winsound.Beep(freq1, duration1)
    winsound.Beep(freq2, duration2)
    time.sleep(1)

CMS-B0001-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-000-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-001-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-002-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-003-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-004-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-005-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-006-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-007-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-008-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-009-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-010-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-011-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-012-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-013-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-014-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-015-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-016-alarmsTagged.csv SAVED
Subset File Ver/CMS-B0001-017-alarmsTagged.csv SAVED
Subset File V